<a href="https://colab.research.google.com/github/am-SaadSiddiqui/rad.ai/blob/main/fraccture_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
generator = ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.2
)

In [ ]:
Train_generator = generator.flow_from_directory(
    "/content/drive/MyDrive/Dataset_NexusHackathon",
    target_size = (224, 224),
    batch_size = 16,
    class_mode ="binary",
    subset = "training"
)

Validation_generator = generator.flow_from_directory(
    "/content/drive/MyDrive/Dataset_NexusHackathon",
    target_size = (224, 224),
    batch_size = 16,
    class_mode ="binary",
    subset = "validation"
)

Found 874 images belonging to 2 classes.
Found 218 images belonging to 2 classes.


In [ ]:
Train_generator.next()[0].shape

(16, 224, 224, 3)

In [ ]:
Validation_generator.next()[0].shape

(16, 224, 224, 3)

In [ ]:
from tensorflow import keras

In [ ]:
base_model = keras.applications.Xception(
    weights=None,
    input_shape=(224, 224, 3),
    include_top=False)

In [ ]:
base_model.trainable = True

In [ ]:
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=True)
x = keras.layers.GaussianNoise(0.25)(x)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(512,activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.GaussianNoise(0.25)(x)
x = keras.layers.Dropout(0.25)(x)
outputs = keras.layers.Dense(1, activation = "sigmoid")(x)
model = keras.Model(inputs, outputs)

In [ ]:
from keras import callbacks

In [ ]:
earlyStopping = keras.callbacks.EarlyStopping(
    patience=7,
)

checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [ ]:
model.fit(Train_generator, epochs=20,
          callbacks=[earlyStopping,model_checkpoint_callback ],
          validation_data=Validation_generator)

Epoch 1/20
55/55 [==============================] - ETA: 0s - loss: 0.5201 - accuracy: 0.8169

55/55 [==============================] - 90s 818ms/step - loss: 0.5201 - accuracy: 0.8169 - val_loss: 0.1856 - val_accuracy: 1.0000
Epoch 2/20
55/55 [==============================] - 14s 261ms/step - loss: 0.1490 - accuracy: 0.9908 - val_loss: 0.0667 - val_accuracy: 1.0000
Epoch 3/20
55/55 [==============================] - 15s 269ms/step - loss: 0.0612 - accuracy: 0.9977 - val_loss: 0.0340 - val_accuracy: 1.0000
Epoch 4/20
55/55 [==============================] - 15s 270ms/step - loss: 0.0352 - accuracy: 0.9989 - val_loss: 0.0588 - val_accuracy: 1.0000
Epoch 5/20
55/55 [==============================] - 15s 269ms/step - loss: 0.0200 - accuracy: 0.9989 - val_loss: 0.0259 - val_accuracy: 1.0000
Epoch 6/20
55/55 [==============================] - 15s 268ms/step - loss: 0.0152 - accuracy: 0.9989 - val_loss: 0.0114 - val_accuracy: 1.0000
Epoch 7/20
55/55 [==============================] - 15s 268ms/step - loss: 0.0129 - accuracy: 0.9989 - val_loss: 0.0053 - val_accuracy: 1.0000
Epoch 8/20

In [ ]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.12.0


In [ ]:
model.save("Test Model 2.h5")

In [ ]:
Test_Generator = generator.flow_from_directory(
    "/content/drive/MyDrive/Dataset_NexusHackathon/val",
    target_size = (224, 224),
    batch_size = 16,
    class_mode ="binary",
    subset = "training"
)

Found 1 images belonging to 2 classes.


In [ ]:
model.evaluate(Test_Generator)